In [3]:
from selenium import webdriver
from selenium.webdriver import Keys
from selenium.webdriver.common.by import By
from linkedin_scraper import actions
import re
from pprint import pprint
import pandas as pd


In [4]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)

email = "salhiabdelhamid01@gmail.com"
password = "TEST@TEST11"

actions.login(driver, email, password, timeout=30)  # if email and password isnt given, it'll prompt in terminal

In [7]:
for i in range(0, 10):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)

In [8]:
posts_content_list = driver.find_elements(By.XPATH, "//div[@data-id]")
filtered_posts = {}
print(f"The number of posts treated is: {len(posts_content_list)}")

The number of posts treated is: 28


In [5]:
def calculate_weight(keywords_list, dict_key):
    for key in keywords_list:
        if key in normalized_post:
            post_dict[dict_key] += 1


In [6]:
prerequested_keywords = ["python", "java", "jee", "j2e", "spring", "angular", "data science", "data analyst", "data scientist", "data engineer", "jupyter"]
exclude_keywords = ["يهود", "صهيوني", "deces", "cancer", "depression", "malaise", "malade", "maladie", "sick", "israel", "palestine", "feel", "pain", "trick", "astuce", "course", "cours", "i'm glad", "i am glad", "i would love", "book", "I would like to thank", "remercier", "big thank", "quotidien", "B2B", "marketing", "paid", "trial", "stage", "pfe", "soutenance", "erreur", "famille", "amis", 
                    "family", "friends", "soutien", "thesis", "gratitude", "grateful",
                    "supervisor", "support", "at least 4", "at least 5", "at least 6", "at least 7", "at least 8",
                    "8 ans", "7 ans", "6 ans", "5 ans", "4 ans", "8ans", "7ans", "6ans", "5ans", "4ans",
                     "8 year", "7 year", "6 year", "5 year", "4 year", "8year", "7year", "6year", "5year", "4year", "préten",
                   "conseil", "les recruteurs", "immense", "suis ravi", "finance", "business", "pleased to", "profonde"]
is_hiring = ["@", "resume", "cv", "recrutement", "join", "rejoin", "recrute", "ressource", "recruit", "hiring", "hire", "job", "work", 
             "hr", "talent", "aquisition", "human", "fresher", "jobopening", "opening", "seek",
             "postule", "challenge", "role", "reach", "contrat", "cdi", "cdd", "freelance",
             "application", "condidature", "resource", "humain", "recherche", "look", "offre", "offer", "opening",
             "employment", "opportunit", "vacanc", "career", "position", "emploi", "poste", "carriere", "embauche"]
is_first_priority = ["junior", "jr", "jnr", "fraichement", "young", "graduated", "graduates", "entry"]

python_keywords = ["beautifulsoup", "bs4", "playwright", "openpyxl", "scraping", "scrap", "pandas",
                   "pycharm", "tkinter", "django", "flask", "data",
                   "jupyter", "analysis"]
java_keywords = ["spring", "boot", "angular", "hibernate", "hql"]
common_keywords = ["css", "html", "ingenieur", "engineer", "excel", "backend", "back end", "back-end", "frontend", "front end", "front-end", "selenium", "web", "api", "security", "keycloack", "developer", "full",
                   "stack", "developpeur", "git", "program", "automation", "automatisation", "data", "données",
                   "software", "scrum", "agile", "oracle", "twilio", "tequila", "sql", "docker", "ci", "cd", "qa"]


In [7]:
def no_excluded(post):
    """if no keyword from the excluded list is present in the post, it returns True"""
    for exclude_key in exclude_keywords:
        if exclude_key in post:
            return False
    return True

def is_today(post):
    """checks the post if it is posted today and ignores sponsored posts"""
    # Define the pattern using regular expression
    pattern = r'\b\d{1,2}[mh]\b'  # Matches one to three digits followed by m or h

    # Use re.search() to find the pattern in the text
    match = re.search(pattern, post.text.split(" •")[0])

    # If there is a match, return True, else return False
    if match:
        return True
    else:
        return False

In [8]:
post_urls = []

for post in posts_content_list:
    normalized_post = post.text.lower().replace("é", "e").replace("è", "e").replace("ù", "u").replace("û", "u") \
        .replace("î", "i")
    for prerequested_keyword in prerequested_keywords:
        post_dict = {"hiring_weight": 0, "priority_weight": 0, "python_weight": 0,
                     "java_weight": 0, "common_weight": 0, "post_url": ""}
        if prerequested_keyword in normalized_post and no_excluded(normalized_post) and is_today(post):
            # print(f"https://www.linkedin.com/feed/update/{post.get_attribute('data-id')}/")
            # todo: check weither the url is already exists or not
            post_url = f"https://www.linkedin.com/feed/update/{post.get_attribute('data-id')}/"
            if post_url not in post_urls:
                calculate_weight(is_hiring, "hiring_weight")
                calculate_weight(is_first_priority, "priority_weight")
                calculate_weight(python_keywords, "python_weight")
                calculate_weight(java_keywords, "java_weight")
                calculate_weight(common_keywords, "common_weight")
                post_dict["post_url"] = post_url
                post_urls.append(post_url)
                try:
                    filtered_posts[prerequested_keyword]["posts"].append(post_dict)
                except KeyError:
                    filtered_posts[prerequested_keyword] = {"posts": [post_dict]}
# todo: check if not promoted then filter only those with m for minutes and h for hours

In [9]:
# make counts 
for key in filtered_posts.keys():
    
    filtered_posts[key]["count"] = len(filtered_posts[key]["posts"])
pprint(filtered_posts)

{}


In [10]:
flattened_data = []
for category, info in filtered_posts.items():
    count = info['count']
    for post in info['posts']:
        hiring_weight = post['hiring_weight']
        priority_weight = post['priority_weight']
        python_weight = post['python_weight']
        java_weight = post['java_weight']
        common_weight = post['common_weight']
        post_url = post['post_url']
        flattened_data.append([category, count, hiring_weight, priority_weight, python_weight, java_weight, common_weight, post_url])

# Create a pandas DataFrame
columns = ['category', 'count', 'hiring_weight', 'priority_weight', 'python_weight', 'java_weight', 'common_weight', 'post_url']
df = pd.DataFrame(flattened_data, columns=columns)

# Print the resulting DataFrame
print(df)

Empty DataFrame
Columns: [category, count, hiring_weight, priority_weight, python_weight, java_weight, common_weight, post_url]
Index: []


In [11]:
sorted_df = df.sort_values(by=['category', 'hiring_weight', 'priority_weight', 'python_weight', 'java_weight', 'common_weight'], ascending=[True, False, False, False, False, False])
print(sorted_df)

Empty DataFrame
Columns: [category, count, hiring_weight, priority_weight, python_weight, java_weight, common_weight, post_url]
Index: []


In [12]:
sorted_df.to_excel('C:\\Users\\Administrator\\Desktop\\sorted_data.xlsx', index=False)

In [13]:
# todo: categorize weights and multiply depending on importance
post_urls = []
filtered_posts = {}
normalized_post = "#hiring nous recrutons un developpeur python junior jr jnr"
for prerequested_keyword in prerequested_keywords:
    post_dict = {"hiring_weight": 0, "priority_weight": 0, "python_weight": 0,
                 "java_weight": 0, "common_weight": 0, "post_url": ""}
    if prerequested_keyword in normalized_post and no_excluded(normalized_post):
        # print(f"https://www.linkedin.com/feed/update/{post.get_attribute('data-id')}/")
        # todo: check weither the url is already exists or not
        post_url = f"https://www.linkedin.com/feed/update/"
        if post_url not in post_urls:
            calculate_weight(is_hiring, "hiring_weight")
            calculate_weight(is_first_priority, "priority_weight")
            calculate_weight(python_keywords, "python_weight")
            calculate_weight(java_keywords, "java_weight")
            calculate_weight(common_keywords, "common_weight")
            post_dict["post_url"] = post_url
            post_urls.append(post_url)
            try:
                filtered_posts[prerequested_keyword]["posts"].append(post_dict)
            except KeyError:
                filtered_posts[prerequested_keyword] = {"posts": [post_dict]}
pprint(filtered_posts)

{'python': {'posts': [{'common_weight': 1,
                       'hiring_weight': 1,
                       'java_weight': 0,
                       'post_url': 'https://www.linkedin.com/feed/update/',
                       'priority_weight': 3,
                       'python_weight': 0}]}}
